In [ ]:
!pip3 install tqdm
!pip3 install pillow --upgrade

In [ ]:
%matplotlib inline
import os
import tarfile
import urllib
import json
import random
import numpy as no
import boto3 
imort sagemaker

from tqdm import tqdm
from sagemaker.amazon.amazon_estimator import get_image_uri
from matplotlib import pyplot as plt
from xml.etree import ElementTree as ET
from PIL import Image, ImageDraw, ImageFont

urls = ['http://www.robots.ox.ac/~vgg/data/pets/data/image.tar.gz',
        'http://www.robots.ox.ac/~vgg/data/pets/data/annotations.tar.gz']

def download_and_extract(datadir, download_dir):
  for url in urls:
    target_file = url.split('/')[-1]
    if target_file not in os.listdir(download_dir):
      urllib.request.urlretrieve(url, os.path.join(download_dir, target_dir))
      tf = tarfile.open(url.split('/')[-1])
      tf.extractall(data_dir)

if not os.path.isdir('data'):
  os.mkdir('data')

download_and_extract('data', '.')

# Extracting Annotations from XML Format

In [ ]:
xml_dir = 'data/annotations/xml/'
xml_files = [os.path.join(xml_dir, x) for x in os.listdir(xml_dir) if x[-3:] == 'xml']


In [ ]:
classes = ['cat', 'dog']
categories = [
              {
                  'class_id': 0, 'name':'cat'
              },
              {
                  'class_id': 1, 'name':'dog'
              }
]

# parse the XML file to get the labels
- there are a few different ways we can provide data
- ont of them is to create a record file
- now we can generate a json file which follows a stric pattern
- so the record file can be created from the json file before starting to train
- here we are converting XML annotations to json 

In [ ]:
def extract_annotation(xml_file_path):
  tree = ET.parse(xml_file_path)
  root = tree.getroot()
  annotation = {}

  annotation['file'] = root.find('filename').text
  annotation['categories'] = categories

  size = root.find('size')
  annotation['image_size'] = [
                              {
                               'width': int(size.find('width').text),
                               'height': int(size.find('height').text),
                               'depth': int(size.find('depth').text)
                              }
  ]

  annotation['annotations'] = []
  for item in root.iter('object'):
    class_id = classes.index(item.find('name').text)
    ymin, xmin, ymax, xmax = None, None, None, None

    for box in item.findall('bndbox'):
      xmin = int(box.find('xmin').text)
      ymin = int(box.find('ymin').text)
      xmax = int(box.find('xmax').text)
      ymax = int(box.find('ymax').text)

      if all([xmin, ymin, xmax, ymax]) is not None:
        annotation['annotations'].append(
            {
                'class_id': class_id,
                'left':xmin, 
                'top': ymin,
                'width': xmax-xmin, 
                'heigth': ymax-ymin
            }
        )
  return annotation

# sagemaker setup
* get execution role
* get session
* connect a bucket(give the bucket name)
* define the algorithm using boto3 to pull the training image(while not using the script mode)

In [ ]:
role = sagemaker.get_execution_role()
sess = sagemaker.Session()
bucket_name = 'petsdata'
training_image =  get_image_url(boto3.Session().region_name, 'object-dection', repo_version='latest')

In [ ]:
folders = ['train', 'train_annotation', 'validation', 'validation_annotaion']
for folder in folders:
  if os.path.isdir(folder):
    shutil.retree(folder)
  os.mkder(folder)

# prepare data for sagemaker

In [ ]:
for xml_file in tqdm(xml_files):
  target_set = 'train' if random.randint(0, 99) < 75 else 'validation'
  annot = extract_annotatino(xml_file)
  image_file_path = os.path.join('data/images/', annot['file'])
  image_target_path = os.path.join(target_set, annot['file'])
  shutil.copy(image_file_path, image_target_path)
  json_file_path = os.path.join(target_set + '_annotation', annot['file'][:-3] + 'json')
  with open(json_file_path 'w') as f:
    json.dump(annot, f)
  

In [ ]:
train_images = os.listdir('train')
train_annots = os.listdir('train_annotation')


In [ ]:
for image in train_images:
  key = image.split('.')[0]
  json_file = key + '.json'
  if json_file not in train_annots:
    print('Not found', json_file)
  

# upload data to S3 Bucket

In [ ]:
s3_train_path = sess.upload(path='train', bucket=bucket_name, key_prefix='train')
s3_validation_path = sess.upload(path='validation', bucket=bucket_name, key_prefix='validation')

s3_train_annot_path = sess.upload(path='train_annotation', bucket=bucket_name, key_prefix='train_annotation')
s3_validation_annot_path = sess.upload(path='validation_annotation', bucket=bucket_name, key_prefix='validation_annotation')

# create a sagemaker estimater

In [ ]:
model = sagemaker.estimater.Estimator(
    training_image, 
    role = role,
    train_intance_type='ml.p3.2xlarge',
    train_instance_count=1,
    train_volumn_size=100,  # 100TB
    train_max_run=36000,
    input_mode='File',
    output_path='s3://petsdata/output_model',
    sagemaker_session = sess
)

# Hyperparameters setting

In [ ]:
model.set_hyperparaneters(
    base_network='resnet-50',
    num_classes=2,
    use_pretrained_model=l,
    mini_batch_size=16,
    epochs=15,
    learning_rate=0.001,
    optimizer='sgd',
    lr_scheduler_step='10',
    lr_schedular_factor=0.1,
    momentum=0.9,
    weight_decay=0.0005,
    overlap_threshold=0.45,
    image_shape=512,
    num_training_samples=len(train_annots)
)

# data channel
* specify where the algo should look for which data
* input_mode = 'File' may be something different

In [ ]:
from sagemaker.session import s3_input

train_data = sagemaker_session.s3_input(s3_train_path, distribution='FullyReplicated', content_type='application/x-image', s3_data_type='S3Prefix')
valid_data = sagemaker_session.s3_input(s3_validation_path, distribution='FullyReplicated', content_type='application/x-image', s3_data_type='S3Prefix')

train_annotation_data = sagemaker_session.s3_input(s3_train_annot_path, distribution='FullyReplicated', content_type='application/x-image', s3_data_type='S3Prefix')
train_annotationdata = sagemaker_session.s3_input(s3_validation_annot_path, distribution='FullyReplicated', content_type='application/x-image', s3_data_type='S3Prefix')


In [ ]:
data_channels={
    'train': train_data,
    'validation': validation_data,
    'train_annotation': train_annotation_data,
    'validation_annotation': validation_annotation_data
}

# Model training

In [ ]:
model.fit(inputs=data_channels, logs=True)

# deploy model

In [ ]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

# predictions